In [1]:
%matplotlib inline
import matplotlib.pyplot as plt, seaborn as sn, mpld3
import pandas as pd, os, glob
sn.set_context('notebook')

# ECOREG

## Final processing of discharge data

Previous notebooks have described an initial exploration of the ECOREG flow data. Susi and Jannicke have now processed the PB and MZB datasets and have decided which sites and time periods are suitable for use in the final analysis. This notebook calculates the IHA parameters for these sites. Note the following:

 1. We want to calculate IHA parameters for **1 year** and **5 years** prior to the ecological sampling. <br><br>
 
 2. There are **21** German sites suitable for the MZB analysis and **25** German sites suitable for PB analysis. All **40** Norwegian sites are suitable for both. <br><br>
 
 3. We want to calculate IHA parameters based on both the **absolute** flow values and on values expressed **relative** to the mean flow over 1 or 5 years. Using relative values will help to remove the effects of stream size on the statistical analysis.

Most of the code here is only slightly modified from previous notebooks, so the comments here are fairly brief.
 
### 1. Get German sites of interest

The table below provides site names, codes and sampling dates for the 25 PB and 21 MZB sites.

In [2]:
# Get sampling/survey times for German sites

# File paths
ger_sites_xls = (r'\\niva-of5\osl-userdata$\JES\Documents\James_Work\Staff\Susi_S\ECOREG'
                 r'\Raw_Data\Germany\Susi_Selected_German_Sites.xlsx')

ger_id_xls = (r'\\niva-of5\osl-userdata$\JES\Documents\James_Work\Staff\Susi_S\ECOREG'
              r'\Raw_Data\Germany\sites_and_ecol.xlsx')

# Read data
ger_sites = pd.read_excel(ger_sites_xls, sheetname='summary sites')
ger_ids = pd.read_excel(ger_id_xls, sheetname='overview')

# Join
ger_sites = pd.merge(ger_sites, ger_ids, how='left', left_on='ID_RS', right_on='ID_RS')

# Get columns of interest and rename
ger_sites = ger_sites[['ID_RS', 'Site', 'date PB', 'date MZB', 'suitable for MZB analyses']]
ger_sites.columns = ['site', 'name', 'pb', 'mzb', 'use_mzb']

ger_sites

,site,name,pb,mzb,use_mzb
0,107000582,altenburg,2009-09-13 15:00:00,2009-06-02,x
1,107000628,gemuend,2009-09-14 15:00:00,2009-05-04,x
2,107000671,kornelimuenster,2007-10-21 15:00:00,2007-05-31,x
3,107001160,oberagger,2009-08-20 15:00:00,2009-04-07,x
4,107001168,rebbelroth,2006-07-26 15:00:00,2006-07-14,x
5,107001228,lohmar,2012-08-23 15:00:00,2012-08-24,x
6,107001230,nespen,2006-09-05 15:00:00,2006-06-12,x
7,107001358,broel,2006-09-05 15:00:00,2006-08-21,x
8,107001436,geisbach,2008-05-07 15:00:00,2008-04-01,x
9,107001723,morsbach,2008-07-14 15:00:00,2008-05-13,x


### 2. Get Norwegian sites of interest

The table below shows site codes, names and sampling dates for the 40 Norwegian sites. Note that in the original Excel file there are some errors in the site codes, which I've fixed in order to link the ecological data to the flows data:

 * The correct site code for Gryta is 6.10 (originally labelled 6.1 in the *flows* dataset) <br><br>
 
 * The correct codes for Høel and Bjoreio are 50.11 and 50.13, respectively (origianlly 5.11 and 5.13 in *ECOREG_WP1_macroinvertebrate data.xlsx*).

In [3]:
# Get Norway sites and sampling times

# File path
nor_sites_xls = (r'\\niva-of5\osl-userdata$\JES\Documents\James_Work\Staff\Susi_S\ECOREG'
                 r'\Raw_Data\Norway\ECOREG_WP1_macroinvertebrate data.xlsx')

# Read data
nor_sites = pd.read_excel(nor_sites_xls, sheetname='Field data')

# Get columns of interest and rename
nor_sites = nor_sites[['site', 'Site name', 'Date']]
nor_sites.columns = ['site', 'name', 'date']

nor_sites

,site,name,date
0,S6.9,Maridalsvatn,2013-09-02
1,S6.10,Gryta,2013-09-02
2,S8.2,Bjørnegårdsvingen,2013-09-03
3,S16.128,Austbygdåi,2013-09-03
4,S16.132,Gjuvå,2013-09-03
5,S16.155,Sønnlandsvatn,2013-09-04
6,S16.1,Omnefoss,2013-09-04
7,S16.193,Hørte,2013-09-04
8,S16.51,Hagadrag,2013-09-05
9,S19.72,Jørundland,2013-09-05


### 3. Read flows data

The code below reads the flows data for all time periods and calculates daily average discharges for each site of interest.

In [4]:
# Define function to perform parsing and resampling

def read_resample_flows(file_name, 
                        site_name,
                        skiprows=11,
                        sep=';',
                        decimal=',',
                        index_col=False,
                        dt_format='%d.%m.%Y %H:%M:%S',
                        freq='M'):
    """ Reads flows data and resamples to the specified frequency.
    
    Args:
        file_name  File to parse
        site_name  Name of site
        skiprows   Number of rows to skip at start
        sep        Column separator
        decimal    Decimal separator
        dt_format  String specifying date format
        freq       Resampling frequency. 'D'=daily; 'M'=monthly; 'A'=Annual
    
    Returns:
        Data frame.
    """
    df = pd.read_csv(file_name, 
                     skiprows=skiprows,
                     header=None,
                     names=['Date_Time', 'Q_m3/s'],
                     index_col=False,
                     sep=sep,
                     decimal=decimal) 
    
    # Parse dates
    df.index = pd.to_datetime(df['Date_Time'], format=dt_format)
    del df['Date_Time']
    
    # Resample
    df = df.resample(freq).mean()
    df.reset_index(inplace=True)
    
    # Add site name as extra column
    df['Site'] = site_name
    df = df[['Site', 'Date_Time', 'Q_m3/s']]
    
    return df

#### 3.1. German time series

In [5]:
# Parse German flows (all data)

###############################################################################
# User input
in_fold = r'\\niva-of5\osl-userdata$\JES\Documents\James_Work\Staff\Susi_S\ECOREG\Raw_Data\Germany'

# Frequency for resampling
freq = 'D'
###############################################################################
  
# Get list of flow files to process
search_path = os.path.join(in_fold, 'Flows', '*')
file_list = glob.glob(search_path)

# Read files
df_list = []
for file_name in file_list:
    # Get site name
    site_name = os.path.split(file_name)[1].split('_')[0]
    
    # Decide whether we need to process this site
    if site_name in ger_sites['name'].values:
    
        # Process differently according to file extension and file formatting
        if site_name == 'hagen-eckesey':
            # This site has a different date format to the other CSV files
            df = read_resample_flows(file_name,
                                     site_name,
                                     skiprows=11,
                                     sep=';',
                                     decimal='.',
                                     dt_format='%d.%m.%Y %H:%M',
                                     freq=freq)        
            df_list.append(df)

        elif file_name[-3:] == 'csv':
            # The rest of the CSV files are consistent
            df = read_resample_flows(file_name,
                                     site_name,
                                     skiprows=11,
                                     sep=';',
                                     decimal=',',
                                     dt_format='%d.%m.%Y %H:%M:%S',
                                     freq=freq)        
            df_list.append(df)

        elif file_name[-3:] == 'zrx':
            # The ZRX files are also consistent
            df = read_resample_flows(file_name,
                                     site_name,
                                     skiprows=5,
                                     sep=' ',
                                     decimal='.',
                                     dt_format='%Y%m%d%H%M%S',
                                     freq=freq)        
            df_list.append(df)

        else:
            raise ValueError('Unexpected file types found in flows data folder.')

# Concatenate results
ger_df = pd.concat(df_list, axis=0)

# Pivot
ger_df = ger_df.pivot(index='Date_Time', columns='Site', values='Q_m3/s')

ger_df.head()

Site,altenburg,beddelhausen,bredelar,broel,bueren,geisbach,gemuend,hagen-haspe,herrntrop,kornelimuenster,...,oberagger,opladen,rebbelroth,rueblinghausen,stephansohl,walkmuehle,weidenau,weidenau2,weine,welda
Date_Time,,,,,,,,,,,,,,,,,,,,,
2002-11-01,26.871875,18.268708,1.877594,3.672760,0.969760,0.415208,4.621177,2.431885,1.921260,0.415573,...,0.290875,18.727771,1.718737,2.086000,1.908448,0.494516,3.243656,3.235000,1.692729,3.491375
2002-11-02,23.022760,16.494115,2.390760,9.018885,1.489625,2.006896,11.703562,6.462062,2.376573,0.934281,...,0.891073,31.387135,6.230521,5.112292,4.790135,0.496000,5.062771,7.806500,2.939740,4.934354
2002-11-03,25.203240,28.141208,2.688115,14.343844,1.991990,2.754521,16.447729,6.961010,2.508802,2.005365,...,1.120167,33.298771,7.345625,6.892604,5.405156,0.499000,6.586396,9.629219,3.665646,6.419531
2002-11-04,24.294562,31.574656,3.038625,14.155792,2.095510,1.763750,13.846292,6.824437,2.881615,1.529271,...,1.021458,30.684490,6.595417,7.198438,5.168510,1.040302,7.885073,9.769125,3.546250,6.536563
2002-11-05,23.682260,28.632313,3.002312,9.719073,1.887323,1.209469,11.142406,6.078385,2.869292,1.328865,...,0.676135,25.387385,4.415521,5.395000,3.741135,1.440000,6.802365,7.782625,2.976948,5.927667


#### 3.2. Norwegian time series

In [6]:
# Parse Norwegian flows (all data)
nor_xls = r'\\niva-of5\osl-userdata$\JES\Documents\James_Work\Staff\Susi_S\ECOREG\Raw_Data\Norway\ECOREG discharge complete.xlsx'
nor_df = pd.read_excel(nor_xls, 
                       sheetname='discharge ECOREG',
                       index_col=0)

# Resample to daily
nor_df = nor_df.resample('D').mean()

# Interpolate
nor_df['S12.8'].interpolate(method='linear', inplace=True)

nor_df.head()

,S109.2,S109.21,S109.9,S12.137,S12.2,S12.207,S12.7,S12.8,S16.1,S16.128,...,S27.16,S30.8,S35.2,S36.31,S36.32,S50.11,S50.13,S6.10,S6.9,S8.2
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-01,9.47282,0.51892,3.59022,20.14581,6.63797,0.92749,1.96617,NaN,17.23073,1.37648,...,5.29065,NaN,NaN,NaN,0.19093,0.97375,1.29150,0.09012,NaN,2.27335
2000-01-02,7.72643,0.51720,2.67732,19.96283,6.63797,0.95418,1.60709,NaN,16.96029,1.26817,...,10.30134,NaN,NaN,NaN,0.22380,1.00387,1.26581,0.09012,NaN,1.95569
2000-01-03,5.95121,0.50668,2.28116,19.98467,6.63797,0.90896,1.52404,NaN,18.14732,1.18040,...,12.24407,NaN,NaN,NaN,0.28290,1.12578,1.24016,0.09866,NaN,1.73315
2000-01-04,5.95121,0.49346,2.07084,20.27129,6.25267,0.76776,1.51251,NaN,19.49061,1.13079,...,10.02236,NaN,NaN,NaN,0.28754,1.16399,1.21454,0.09866,NaN,1.75367
2000-01-05,5.95121,0.50905,2.08630,19.97185,6.25267,0.87590,1.46788,NaN,21.34705,0.98371,...,6.99554,NaN,NaN,NaN,0.43283,1.14173,1.21454,0.09457,NaN,1.55473


### 4. Calculate IHA parameters

#### 4.1. Generate lookup table between site codes and names

In [7]:
# Extract codes and names
ger_codes = ger_sites[['site', 'name']]
nor_codes = nor_sites[['site', 'name']]

# Concatenate
codes = pd.concat([nor_codes, ger_codes], axis=0)
codes.index = codes['site']
del codes['site']

codes

,name
site,
S6.9,Maridalsvatn
S6.10,Gryta
S8.2,Bjørnegårdsvingen
S16.128,Austbygdåi
S16.132,Gjuvå
S16.155,Sønnlandsvatn
S16.1,Omnefoss
S16.193,Hørte
S16.51,Hagadrag


#### 4.2. Function to pass time series to IHA pachage in R

In [8]:
def process_iha(df):
    """ Processes the first five groups of IHA indicators.
    
    Args:
        df  Pandas data frame with DAILY resolution consisting
            of a single column entitled 'site_name' and a date-time
            index.
    
    Returns:
        Pandas data frame of IHA parameters calculated from the
        IHA R package.
    """
    # Set up connection to R. This all seems unnecessarily complicated!
    import rpy2.interactive as r
    import rpy2.interactive.packages
    from rpy2.robjects.packages import importr
    import pandas.rpy.common as com
    from rpy2.robjects import pandas2ri
    pandas2ri.activate()
    
    # Circular mean function from scipy (see above)
    from scipy.stats import circmean

    # Load necessary R packages
    importr('zoo', lib_loc="//niva-of5/osl-userdata$/JES/Documents/R/win-library/3.2")   
    importr('IHA', lib_loc="//niva-of5/osl-userdata$/JES/Documents/R/win-library/3.2")
    
    # Import R packages into interactive session
    zoo = r.packages.importr('zoo')
    iha = r.packages.importr('IHA')
    
    # Get path to package methods
    rlib = r.packages.packages

    # Convert df to 2 columns ['Dates', 'Flows']
    df2 = df.reset_index()

    # Convert Pandas df to R 
    ts = rlib.zoo.read_zoo(df2, format="%Y-%m-%d")
    
    # Processing for Group 1
    # Calculate group 1 stats.
    rg1 = rlib.IHA.group1(ts)

    # Convert back to Python
    grp1 = com.convert_robj(rg1)

    # Get stats
    grp1 = grp1.describe().T

    # Coefficient of dispersion
    grp1['CoD'] = (grp1['75%'] - grp1['25%']) / grp1['50%']

    # Format grp 1 df
    grp1.index.name = 'Indicator'
    grp1.reset_index(inplace=True)
    grp1['Group'] = 1
    grp1.index = [grp1['Group'], grp1['Indicator']]
    grp1 = grp1[['50%', 'CoD']]
    
    # Processing for Group 2
    # Calculate group 2 stats.
    rg2 = rlib.IHA.group2(ts)

    # Convert back to Python
    grp2 = com.convert_robj(rg2)

    # Get stats
    grp2 = grp2.describe().T

    # Coefficient of dispersion
    grp2['CoD'] = (grp2['75%'] - grp2['25%']) / grp2['50%']

    # Format grp 2 df
    grp2.index.name = 'Indicator'
    grp2.reset_index(inplace=True)
    grp2['Group'] = 2
    grp2 = grp2[grp2['Indicator'] != 'year']
    grp2.index = [grp2['Group'], grp2['Indicator']]
    grp2 = grp2[['50%', 'CoD']]

    # Processing for Group 3
    # Calculate group 3 stats.
    rg3 = rlib.IHA.group3(ts)

    # Convert back to Python
    grp3 = com.convert_robj(rg3)

    # Get stats using circular mean and assuming 366 days per year (as in IHA)
    c_av = circmean(grp3, high=366, low=0, axis=0)
    
    # We won't include a CoD for this stat.
    # Build df to store this info
    grp3 = pd.DataFrame(data=[c_av, [pd.np.nan, pd.np.nan]], 
                        columns=['Min', 'Max'],
                        index=['50%', 'CoD']).T

    # Format grp 2 df
    grp3.index.name = 'Indicator'
    grp3.reset_index(inplace=True)
    grp3['Group'] = 3
    grp3.index = [grp3['Group'], grp3['Indicator']]
    grp3 = grp3[['50%', 'CoD']]

    # Processing for Group 4
    # Calculate group 4 stats.
    rg4 = rlib.IHA.group4(ts)

    # Convert back to Python
    grp4 = com.convert_robj(rg4)

    # Get stats
    grp4 = grp4.describe().T

    # Coefficient of dispersion
    grp4['CoD'] = (grp4['75%'] - grp4['25%']) / grp4['50%']

    # Format grp 4 df
    grp4.index.name = 'Indicator'
    grp4.reset_index(inplace=True)
    grp4['Group'] = 4
    grp4.index = [grp4['Group'], grp4['Indicator']]
    grp4 = grp4[['50%', 'CoD']]  

    # Processing for Group 5
    # Calculate group 5 stats.
    rg5 = rlib.IHA.group5(ts)

    # Convert back to Python
    grp5 = com.convert_robj(rg5)

    # Get stats
    grp5 = grp5.describe().T

    # Coefficient of dispersion
    grp5['CoD'] = (grp5['75%'] - grp5['25%']) / grp5['50%']

    # Format grp 4 df
    grp5.index.name = 'Indicator'
    grp5.reset_index(inplace=True)
    grp5['Group'] = 5
    grp5.index = [grp5['Group'], grp5['Indicator']]
    grp5 = grp5[['50%', 'CoD']]  

    # Combine results
    iha_res = pd.concat([grp1, grp2, grp3, grp4, grp5], axis=0)
    
    # Rename 50% col (because not all values are actually medians)
    # Also add heirarchical index for site_name
    iha_res.columns = [[df.columns[0], df.columns[0]],['Cent_Est', 'CoD']]
   
    return iha_res

#### 4.3. Loop over sites

In [9]:
# Output file
out_xlsx = r'\\niva-of5\osl-userdata$\JES\Documents\James_Work\Staff\Susi_S\ECOREG\IHA_Params\iha_params.xlsx'
writer = pd.ExcelWriter(out_xlsx)

# Loop over periods, statistics and ecological indices
for per in ['short', 'long']:
    for stat in ['abs', 'rel']:
        for eco in ['pb', 'mzb']:
            # DF to store output
            df_list = []

            # Process Norwegain sites
            print 'Processing %s term %s values for %s...' % (per, stat, eco)
            print '  Norway:'

            for col in nor_df.columns:
                # Print progress
                print '    %s.' % col

                # Define long and short periods according to station
                if per == 'short':
                    yrs = 1
                elif (per == 'long') and (col == 'S25.6'): # Only use 3 years for Homstølvatn
                    yrs = 3
                else: 
                    yrs = 5

                # Get start and end dates
                end = nor_sites[nor_sites['site']==col].iloc[0]['date']
                st = end - pd.DateOffset(years=yrs)

                # Get series and trunctae
                df = nor_df[[col]].truncate(before=st, after=end)

                # Fill no data
                df.interpolate(method='linear', inplace=True)
                
                # Assert no missing values remain
                assert df.isnull().sum(axis=0)[0] == 0, 'Dataframe has missing values'

                # Calculate relative values if necessary
                if stat == 'rel':
                    # "Normalise" by dividing by the mean
                    df = df / df.mean()

                # Append results
                df_list.append(process_iha(df))

            # Process German sites
            print '  Germany:'

            for col in ger_df.columns:
                # Print progress
                print '    %s.' % col

                # Decide whether to process this site
                if ((eco == 'mzb') and (col in ger_sites['name'][ger_sites['use_mzb'].isnull()].values)):
                    pass
                else:
                    # Get the site code
                    site = ger_sites[ger_sites['name']==col].iloc[0]['site']
                    
                    # Define long and short periods
                    if per == 'short':
                        yrs = 1
                    else: 
                        yrs = 5

                    # Get start year and end year for this site
                    end = ger_sites[ger_sites['name']==col].iloc[0][eco]
                    st = end - pd.DateOffset(years=yrs)

                    # Get series and trunctae at specified dates
                    df = ger_df[[col]].truncate(before=st, after=end)

                    # Rename with site code rather than site name
                    df.columns = [site,]
                    
                    # Fill no data
                    df.interpolate(method='linear', inplace=True)

                    # Assert no missing values remain
                    assert df.isnull().sum(axis=0).iloc[0] == 0, 'Dataframe has missing values'

                    # Append results
                    df_list.append(process_iha(df))

            # Combine into final output
            iha_params = pd.concat(df_list, axis=1)

            # Get just the 'Cent_Est' columns
            iha_df = iha_params.xs('Cent_Est', level=1, axis=1).reset_index()
            iha_df.index = iha_df['Indicator']
            del iha_df['Group'], iha_df['Indicator']

            # Convert shape to samples x features
            iha_df = iha_df.T

            # Join in site names and reorder columns
            iha_cols = list(iha_df.columns)
            iha_df = pd.merge(iha_df, codes, how='left', left_index=True, right_index=True)
            iha_df = iha_df[['name',]+iha_cols]
            
            # Write output (ignoring CoD stats. for now)
            iha_df.to_excel(writer, '%s_term_%s_%s' % (per, stat, eco))
writer.save()

Processing short term abs values for pb...
  Norway:
    S109.2.
    S109.21.
    S109.9.
    S12.137.
    S12.2.
    S12.207.
    S12.7.
    S12.8.
    S16.1.
    S16.128.
    S16.132.
    S16.155.
    S16.193.
    S16.51.
    S19.72.
    S2.129.
    S2.267.
    S2.268.
    S2.303.
    S2.32.
    S2.434.
    S2.439.
    S2.479.
    S2.592.
    S2.611.
    S20.2.
    S21.21.
    S25.6.
    S27.13.
    S27.15.
    S27.16.
    S30.8.
    S35.2.
    S36.31.
    S36.32.
    S50.11.
    S50.13.
    S6.10.
    S6.9.
    S8.2.
  Germany:
    altenburg.
    beddelhausen.
    bredelar.
    broel.
    bueren.
    geisbach.
    gemuend.
    hagen-haspe.
    herrntrop.
    kornelimuenster.
    lohmar.
    meschede.
    morsbach.
    nespen.
    niederschelden2.
    oberagger.
    opladen.
    rebbelroth.
    rueblinghausen.
    stephansohl.
    walkmuehle.
    weidenau.
    weidenau2.
    weine.
    welda.
Processing short term abs values for mzb...
  Norway:
    S109.2.
    S109.21.
    S109.9.
 

C:\Data\64_Bit_WinPython\python-2.7.10.amd64\lib\site-packages\ipykernel\__main__.py:17: FutureWarning: The pandas.rpy module is deprecated and will be removed in a future version. We refer to external packages like rpy2. 
See here for a guide on how to port your code to rpy2: http://pandas.pydata.org/pandas-docs/stable/r_interface.html
